## Import libraries and extracts sections

In [0]:
import os
import pandas as pd
os.system('pip install scholarly')
import scholarly
# os.chdir('C:\\Users\\Cafral\\Desktop\\kaggle\\CORD-19-research-challenge\\data_v7')

In [2]:
# Downloading all datasets

!pip install -q kaggle

f = open("kaggle.json", "w")
f.write('{"username":"pranjalya","key":"fa4deb4f89e2fe282a3e29c166413ecf"}')
f.close()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d skylord/coronawhy
!kaggle datasets download allen-institute-for-ai/CORD-19-research-challenge -f metadata.csv
!unzip metadata.csv.zip
!rm -rf metadata.csv.zip
!unzip coronawhy.zip v6_text/*
!wget -O v7_text.json https://storage.googleapis.com/coronawhy/NLPDatasets/v7_preprocessed/v7_text.json

PATH = "v6_text/v6_text/"

100% 12.7G/12.8G [03:26<00:00, 91.1MB/s]
100% 12.8G/12.8G [03:26<00:00, 66.3MB/s]
 33% 9.00M/27.6M [00:00<00:00, 55.2MB/s]
100% 27.6M/27.6M [00:00<00:00, 104MB/s] 
Archive:  metadata.csv.zip
  inflating: metadata.csv            
Archive:  coronawhy.zip
  inflating: v6_text/v6_text/v6_text_0.pkl  
  inflating: v6_text/v6_text/v6_text_1.pkl  
  inflating: v6_text/v6_text/v6_text_10.pkl  
  inflating: v6_text/v6_text/v6_text_11.pkl  
  inflating: v6_text/v6_text/v6_text_12.pkl  
  inflating: v6_text/v6_text/v6_text_13.pkl  
  inflating: v6_text/v6_text/v6_text_14.pkl  
  inflating: v6_text/v6_text/v6_text_15.pkl  
  inflating: v6_text/v6_text/v6_text_16.pkl  
  inflating: v6_text/v6_text/v6_text_17.pkl  
  inflating: v6_text/v6_text/v6_text_18.pkl  
  inflating: v6_text/v6_text/v6_text_19.pkl  
  inflating: v6_text/v6_text/v6_text_2.pkl  
  inflating: v6_text/v6_text/v6_text_3.pkl  
  inflating: v6_text/v6_text/v6_text_4.pkl  
  inflating: v6_text/v6_text/v6_text_5.pkl  
  inflating: v6_t

In [0]:
firstpass = True

for pkl in os.listdir(PATH):
  df = pd.read_pickle(PATH+pkl, compression='gzip')
  if(firstpass):
    v7_data = pd.read_json('v7_text.json')
    df_method = pd.concat([v7_data[v7_data['section']=='methods'], df[df['section']=='methods']])
    df_result = pd.concat([v7_data[v7_data['section']=='results'], df[df['section']=='results']])
    firstpass = False
  else:
    df_method = pd.concat([df_method, df[df['section']=='methods']])
    df_result = pd.concat([df_result, df[df['section']=='results']])

df_method.to_csv('method_df.csv')
df_result.to_csv('result_df.csv')

# Load the data

In [4]:
df_method = pd.read_csv('method_df.csv')
df_result = pd.read_csv('result_df.csv')

print("No of unique papers in method section : ", df_method['paper_id'].nunique(), " out of ", 
      len(df_method), " rows in dataframe")
print("No of unique papers in result section : ", df_result['paper_id'].nunique(), " out of ", 
      len(df_result), " rows in dataframe")

df_method.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


No of unique papers in method section :  9693  out of  188137  rows in dataframe
No of unique papers in result section :  7820  out of  178851  rows in dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188137 entries, 0 to 188136
Data columns (total 38 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   Unnamed: 0                       188137 non-null  int64 
 1   paper_id                         188137 non-null  object
 2   language                         188137 non-null  object
 3   section                          188137 non-null  object
 4   sentence                         188137 non-null  object
 5   lemma                            188137 non-null  object
 6   UMLS                             188137 non-null  object
 7   GGP                              188137 non-null  object
 8   SO                               188137 non-null  object
 9   TAXON                            188137 

# Extracting sentences which contain topic ngrams

In [5]:
def find_ngrams(dataframe, columnToSearch, keywords):
    df_w_ngrams = dataframe[dataframe[columnToSearch].str.contains('|'.join(keywords), case=False) == True]
    
    return df_w_ngrams

population_density_ngrams = ['population density','number of people in','number of people per',
                             'highly populated areas','highly populated countries',
                             'densely populated countries','densely populated areas',
                             'high density areas','high density countries'
                             ,'population densities', 'density of population','sparsely populated',
                             'densely populated','density of the population','dense population',
                             'populated areas', 'densely inhabited','housing density',
                             'densely-populated','concentration of people', 'population pressure',
                             'population studies','populated regions','populous',
                             'high population densities','residential densities','overpopulated']

#Extracting sentences which contain ngrams

df_method_p = find_ngrams(df_method,'sentence',population_density_ngrams)

df_result_p = find_ngrams(df_result,'sentence',population_density_ngrams)

print("There are {} sentences containing keywords/ngrams in Method section.".format(len(df_method_p)))
print("There are {} sentences containing keywords/ngrams in Result section.".format(len(df_result_p)))

# Merging the method and result section sentences into single dataframe
df_real = pd.concat([df_method_p, df_result_p])

print("Total unique papers in Method section : {}".format(df_method_p['paper_id'].nunique()))
print("Total unique papers in Result section : {}".format(df_result_p['paper_id'].nunique()))
print("Total unique papers in combined section : {}".format(df_real['paper_id'].nunique()))

There are 121 sentences containing keywords/ngrams in Method section.
There are 62 sentences containing keywords/ngrams in Result section.
Total unique papers in Method section : 84
Total unique papers in Result section : 39
Total unique papers in combined section : 117


# Keeping all the sentences from papers that had topic ngrams

In [6]:
df_method_all_sentence = pd.merge(df_method[['paper_id','sentence']],df_method_p['paper_id'],on='paper_id',how='right')
df_method_all_sentence.rename(columns={'sentence_x':'all_sentences','sentence_y':'ngram_sentence'},inplace=True)

df_result_all_sentence = pd.merge(df_result[['paper_id','sentence']],df_result_p['paper_id'],on='paper_id',how='right')
df_result_all_sentence.rename(columns={'sentence_x':'all_sentences','sentence_y':'ngram_sentence'},inplace=True)

df_all_sentences = pd.concat([df_method_all_sentence, df_result_all_sentence])
print("Total unique papers in combined section : {}".format(df_all_sentences['paper_id'].nunique()))

Total unique papers in combined section : 117


# Extracting methodolody, sample size, causal nature, sentences refering to coronavirus, fatality

In [0]:
def extract_features(ngramDf,allSentdataFrame):
    # extracting methodology
    methods_list = ['regression','OLS','logistic','time series','model','modelling','simulation',
                    'forecast','forecasting']
    methodology = find_ngrams(allSentdataFrame,'sentence',methods_list)

    #extracting sample size
    sample_size_list = ['population size','sample size','number of samples','number of observations',
                        'number of subjects']
    sample_size = find_ngrams(allSentdataFrame, 'sentence', sample_size_list)

    #extracting nature of correlation
    causal_list =['statistically significant','statistical significance',
                  'correlation','positively correlated','negatively correlated','correlated',
                  'p value','p-value','chi square','chi-square',
                  'confidence interval','CI','odds ratio','OR','coefficient']

    causality_type = find_ngrams(allSentdataFrame, 'sentence', causal_list)

    # extracting coronavirus related sentence #can someone check and update this list?
    coronavirus_list = ['severe acute respiratory syndrome','sars-cov','sars-like',
                        'middle east respiratory syndrome','mers-cov','mers-like',
                        'covid-19','sars-cov-2','2019-ncov','sars-2',
                        'sarscov-2','novel coronavirus','corona virus','coronaviruses',
                        'sars','mers','covid19','covid 19']

    coronavirus = find_ngrams(allSentdataFrame, 'sentence', coronavirus_list)

    # extracting outcome
    disease_stage_list = ['lethal', 'morbid',"death", "fatality", "mortality","lethal", "lethality", "morbidity"]

    fatality = find_ngrams(allSentdataFrame, 'sentence', disease_stage_list)

    df_list = [methodology,sample_size,causality_type,coronavirus,fatality]
    df_list_name = ['methodology','sample_size','causality_type','coronavirus','fatality']
    i=0
    for one_df in df_list:
        one_df.rename(columns={'sentence':df_list_name[i]},inplace=True)
        print(one_df.head())
        grouped_one_df = one_df.groupby('paper_id')[df_list_name[i]].sum()
        ngramDf = pd.merge(ngramDf,grouped_one_df,on='paper_id',how='left')
        i=i+1
    return ngramDf

In [8]:
df_real = extract_features(df_real, df_all_sentences)

                                   paper_id                                        methodology
0  a98b1b317a570c905d9feff5bedd079effe89a7a  Basic transmission dynamics of COVID-19 was mo...
2  a98b1b317a570c905d9feff5bedd079effe89a7a  Basic transmission dynamics of COVID-19 was mo...
4  a98b1b317a570c905d9feff5bedd079effe89a7a  Basic transmission dynamics of COVID-19 was mo...
6  a98b1b317a570c905d9feff5bedd079effe89a7a  Basic transmission dynamics of COVID-19 was mo...
8  dbd4b4192c4b4a84d3b3b6e1f82a56d5351aaa20  The dynamic model (7) is impossible to solve a...
                                      paper_id                                        sample_size
604   6445688d3f59cb7e02f2a8b28450cdf118a8a373  On a population level that means that the tota...
676   955b36f4b007abbfaa28385a627b03e2102313d9  Such an analysis was not feasible in this stud...
1028  873d2ca5817ad916b491680ffc9e3260a8469e60  The population living in urban slums is report...
1043  873d2ca5817ad916b491680ffc9e3260

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.p

# Merge with Metadata

In [9]:
metadata = pd.read_csv('metadata.csv')
metadata.rename(columns={'sha':'paper_id'}, inplace = True)
metadata['paper_id'] = metadata['paper_id'].astype("str")

#Merging the given papers with their metadata
df_real = df_real.merge(metadata[['paper_id', 'title', 'abstract', 'publish_time', 'authors',
                                  'url']], on='paper_id', how='left') #'title_w_ngram','abstract_w_ngram'

#Keeping only the fields which are relevant to us.
df_real = df_real[['paper_id','language', 'section', 'sentence', 'lemma', 'UMLS', 'sentence_id', 
                   'publish_time', 'authors', 'url','methodology','sample_size','causality_type','coronavirus',
                   'fatality','title','abstract','publish_time','authors',
                   'url','TAXON']]#'title_w_ngram','abstract_w_ngram',
df_real.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   paper_id        183 non-null    object
 1   language        183 non-null    object
 2   section         183 non-null    object
 3   sentence        183 non-null    object
 4   lemma           183 non-null    object
 5   UMLS            183 non-null    object
 6   sentence_id     181 non-null    object
 7   publish_time    142 non-null    object
 8   authors         141 non-null    object
 9   url             142 non-null    object
 10  methodology     116 non-null    object
 11  sample_size     46 non-null     object
 12  causality_type  183 non-null    object
 13  coronavirus     40 non-null     object
 14  fatality        34 non-null     object
 15  title           142 non-null    object
 16  abstract        141 non-null    object
 17  publish_time    142 non-null    object
 18  authors   

In [0]:
grouped = df_real.groupby('paper_id')
def keywordcounter(sentences, keywords_list):
    '''
    Input : List of sentences, List of keywords
    Returns : Keywords present in sentences, Total count of all keywords present in Input
    '''
    keyword = {}
    sent = " ".join(sentences)
    for pol in keywords_list:
        counter = sent.lower().count(pol)
        if (counter > 0):
            keyword[pol] = counter
    return list(keyword.keys()), sum(keyword.values())

def aggregation(item,keyWordList,RiskFactor):
    '''
    Input : Dataframe of sentences of a paper
    Return : Datframe in Standard Output format
    '''
    dfo = {}
    
    dfo['Risk Factor'] = RiskFactor
    dfo['Title'] = item['title'].iloc[0]
    dfo['Keyword/Ngram'], dfo['No of keyword occurence in Paper'] = keywordcounter(item['sentence'].tolist(),
                                                                                 keyWordList)
    dfo['paper_id'] = item['paper_id'].iloc[0]
    
    if (item['url'].iloc[0].isnull().any()==False):
        dfo['URL'] = item['url'].iloc[0].tolist()
    else:
        dfo['URL']=''
    #dfo['Sentences from Title']= item['title_w_ngram'].iloc[0]                        
    #dfo['Sentences from Abstract']= item['abstract_w_ngram'].iloc[0]
    dfo['Sentences from Method'] = item[item['section']=='methods']['sentence'].tolist()
    dfo['Sentences from Result'] = item[item['section']=='results']['sentence'].tolist()
    
    if (item['authors'].iloc[0].isnull().any()==False):#(item['authors'].iloc[0].isnull()==False):
        dfo['Authors'] = item['authors'].iloc[0].tolist()
    else:
         dfo['Authors'] = ''
    # For papers which do not have title (not in metadata) we have to resolve exceptions
    try:
        dfo['No of Citations'] = next(scholarly.search_pubs_query(item['title'].iloc[0])).citedby
    except:
        dfo['No of Citations'] = 0
        
    dfo['Correlation'] = item['causality_type'].iloc[0]
    dfo['Design Methodology'] = item['methodology'].iloc[0]
    dfo['Sample Size'] = item['sample_size'].iloc[0]
    dfo['Coronavirus'] = item['coronavirus'].iloc[0]
    dfo['Fatality'] = item['fatality'].iloc[0]
    dfo['TAXON'] =item['TAXON'].iloc[0]
    
    return dfo

df_output = pd.DataFrame(columns=['Risk Factor', 'Title','Keyword/Ngram', 'No of keyword occurence in Paper',
                                  'paper_id', 'URL',
                                  'Sentences from Result', 'Sentences from Method',
                                  'Authors','No of Citations', 'Correlation', 
                                  'Design Methodology', 'Sample Size',
                                 'Coronavirus','Fatality','TAXON'])
for key, item in grouped:
    df_output = pd.concat([df_output, pd.DataFrame([aggregation(item,population_density_ngrams,'Population Density')])])

df_output = df_output.reset_index()
df_output.to_excel('population_density_json.xlsx')

In [11]:
df_output.head()

,index,Risk Factor,Title,Keyword/Ngram,No of keyword occurence in Paper,paper_id,URL,Sentences from Result,Sentences from Method,Authors,No of Citations,Correlation,Design Methodology,Sample Size,Coronavirus,Fatality,TAXON
0,0,Population Density,Distribution and Risk Factors of 2009 Pandemic...,[population density],3,043aa68f2c784899f71225e8eb233150760a6a54,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,[Population density and the density of medical...,[],"[Fang, Li-Qun; Wang, Li-Ping; de Vlas, Sake J....",0,"A total of 121,805 cases of pandemic influenza...",Table 2 shows that all climatic factors (excep...,NaN,NaN,The first death caused by pandemic influenza w...,[]
1,0,Population Density,NaN,[population density],1,0900c2c26866f1fed9afc70835e4df7f453c39c7,,[],[20 The results are not sensitive to using dis...,,0,This section describes the empirical model use...,This section describes the empirical model use...,NaN,We are particularly interested in examining wh...,NaN,[]
2,0,Population Density,Spatiotemporal diffusion of influenza A (H1N1)...,"[population density, populous]",2,0f76bdee678e512d5bfb2d3597ad57a1b5d74bc4,[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC...,"[Population density, municipal road transport,...",[],"[da Costa, Ana Carolina Carioca; Codeço, Cláud...",0,The spatial distribution of influenza A (H1N1)...,Table 1 shows the effects of covariates that w...,NaN,NaN,The actions recommended for this phase were ai...,[]
3,0,Population Density,NaN,[population density],1,13bea4e60e6161f1132a83bc5ed2b3bceab1e05e,,[We established replicate bacterial population...,[],,0,We established replicate bacterial populations...,NaN,NaN,"Conversely, adaptation of zoonotic parasites t...",NaN,"['bacterial', 'host', 'bacteria']"
4,0,Population Density,Natural history and epidemiology of respirator...,[densely populated],1,140c32554663e4033d23a392bd287b0239bfe1d1,[https://doi.org/10.1016/j.vaccine.2015.08.048...,[],[It is located in the low-income and densely p...,"[Halasa, Natasha; Williams, John; Faouri, Sami...",0,"We conducted a prospective, year-round viral s...",NaN,NaN,NaN,NaN,['Amman']
